In [1]:
import numpy as np
import matplotlib.pyplot as plt
##New, for VS code!
%matplotlib widget

In [2]:
class opticalSystem:
    def __init__(self, focalsIn, gapsIn):
        #floats, are what they say, in mms
        #below we set up the above as lists
        self.focal = focalsIn
        self.delX = gapsIn
        self.x = np.cumsum(self.delX)
        ##np.append(self.x, self.x[-1] + 1000)
        #then make a list of the Rates of Convergance of our lenses
        self.c = [1/v for v in focalsIn]
    
    def simRay(self,rayC, y1):
        
        y0 = y1 + (rayC*self.delX[0])
        y = [y0, y1]
        slope = rayC
        for i in range(0,len(self.c)-1):
            slope = slope + self.c[i]*y[-1]
            y.append(y[-1] - slope*self.delX[i])
            
        x = np.array(self.x)
        
        return np.array([x,y])

In [3]:
def intersection(x1,y1,x2,y2):
    #this takes the ray data I'm building up and returns any intersections between the two rays.
    #all inputs are lists of points that are connected by lines
    #IT IS ASSUMED ALL INPUT LISTS ARE OF SAME LENGTH, x1 = x2
    
    intersects = []
    #find intersections in the between lens regime
    for i in range(1,len(x1),1):
        denom = y1[i-1]-y1[i]-y2[i-1]+y2[i]
        
        if denom != 0:
            intX = (y1[i-1]-y2[i-1]) * (x1[i]-x1[i-1]) / denom
            intX += x1[i-1]
            intY = y2[i-1] + (intX-x2[i-1]) * (y2[i]-y2[i-1])/(x2[i]-x2[i-1])
            if intX <= x1[i] and intX >= x1[i-1]:
                intersects.append([intX, intY])
                
    
    #find any after lens intersections
    
    
    return np.array(intersects)

In [4]:
def imageRays(distance,yIn,lensRadius,numRays):
    #This function generates a list of indident rays defined by an objects location.
    #all in are floats except numRays, which is an int
    #outputs a numpy array of the rays slopes and x hit on a lens
    rays = []
    
    for i in np.linspace(-lensRadius,lensRadius,num=numRays):
        c = (yIn-i)/distance
        y = i
        rays.append(np.array([c,y]))
    
    
    return np.array(rays)

In [5]:
#Now we start exploring some rapid simulating, to see if I can't find a perfect solution. NBD

Focals = [-135, 0.0000000001, 67, 70, 80, 100, 125, 150] #truncated
#[-643, -570, -135, 0.0000000001, 67, 70, 73, 90, 100, 108, 115, 121, 125, 127, 130, 133, 140, 143, 150, 160, 172, 175, 245] #updated 3/26/2023
#[205,-570,130,133,125,135,127,93,121,175,-135,115,108,172,140,160,245,100,90,70,67]
Gap1s = [0,5,15,30,40,50,70,80,100,150,200]
#[0,2,4,6,10,15,20,25,30,35,40,45,50,60,70,80,100,120,140,175,200]
Gap2s = Gap1s

fit = 1
tested = 1

#This goes through every combo of above perameters
data = []
rays = imageRays(99999,1000,10,2)

for focal1 in Focals:
    for focal2 in Focals:
        for focal3 in Focals:
            for focal4 in Focals:
                for gap1 in Gap1s:
                    for gap2 in Gap2s:
                        for gap3 in Gap1s:
                            #set up lens system object
                            lens1 = opticalSystem([focal1, focal2, focal3,  focal4], [100, gap1, gap2, gap3, 1000])
                            x = []
                            y = []
                            
                            #pass incoming rays into lens system object
                            for i in range(0,len(rays[:,0]),1):
                                tracedRay = lens1.simRay(rays[i,0],rays[i,1])
                                x.append(tracedRay[0])
                                y.append(tracedRay[1])
                            
                            #find intersections of these rays
                            images = intersection(x[0],y[0],x[1],y[1])
                            #print(images)
                            #save settup data if we get any images at all
                            inner = []
                            outer = []
                            for image in images:
                                if image[0]<(gap1+gap2) and image[0] > gap1:
                                    inner = image
                                elif image[0] > (gap1+gap2):
                                    outer = image
                                    
                            if inner != [] and outer != []:
                                #find theoretical center
                                rays2 = np.array([(1,-2),(-1,2)])
                                x = []
                                y = []
                                for j in [0,1]:
                                    tracedRay = lens1.simRay(rays2[j,0],rays2[j,1])
                                    x.append(tracedRay[0])
                                    y.append(tracedRay[1])

                                intersect = intersection(x[0],y[0],x[1],y[1])
                                if len(intersect) > 1:
                                    center = intersect[1,:]
                                else:
                                    center = np.array([0,0])
                                #save data to array
                                data.append([inner[0], inner[1], outer[0], outer[1], focal1, focal2, focal3, focal4, gap1, gap2, gap3, center[0], center[1]])
                                
                                fit += 1
                            tested += 1
                        
print("I tested " + str(tested) + " lens setups and found " + str(fit) + " that satisfied image location requirments.")
name = "compoundLensTest"
np.save(name, data, allow_pickle=True, fix_imports=True)
print("Saved Data to " + name)


C:\Users\Levi Rhoden\AppData\Local\Temp\ipykernel_6216\1421255217.py:14: RuntimeWarning: invalid value encountered in scalar divide
  intY = y2[i-1] + (intX-x2[i-1]) * (y2[i]-y2[i-1])/(x2[i]-x2[i-1])
C:\Users\Levi Rhoden\AppData\Local\Temp\ipykernel_6216\4235005299.py:47: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if inner != [] and outer != []:


I tested 5451777 lens setups and found 173537 that satisfied image location requirments.
Saved Data to compoundLensTest
